In [1]:
import pandas as pd
import numpy as np
from docplex.mp.model import Model

np.random.seed(42)

## Mathematical Model

#### Sets
- C	Set of campaigns.
- U	Set of customers.
- H	Set of channels
- D	Set of planning days.
- I	Set of quota categories.
- P	Set of priority categories.


In [2]:
C = 10 # number of campaigns
U = 10000 # number of customers.
H = 3 # number of channels.
D = 7 # number of planning days.
I = 3 # number of quota categories.
P = 10 # number of priority categories.

#### Parameters

##### - eligibility
$$
e_{cu}\left\{\begin{array}\\
        1 & \mbox{if }  customer\ u\ is\ eligible\ for\ campaign\ c\\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [3]:
e_cu = np.random.choice(2,(C, U))
#e_cu = np.ones((C, U), dtype='int8')
e_cu

array([[0, 1, 0, ..., 1, 1, 0],
       [0, 1, 0, ..., 0, 1, 0],
       [0, 1, 1, ..., 1, 1, 0],
       ...,
       [0, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 1],
       [1, 1, 0, ..., 0, 1, 1]])

##### - quota categories
$$
q_{ic}\left\{\begin{array}\\
        1 & \mbox{if }  campaign\ c\ is\ a\ i^{th} type\ quota\ category\ campaign\ \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [4]:
q_ic = np.random.choice(2, (I,C))
#q_ic = np.zeros((I,C), dtype='int8')
q_ic

array([[0, 1, 1, 1, 1, 1, 0, 0, 0, 1],
       [0, 1, 1, 0, 1, 0, 0, 1, 0, 0],
       [1, 0, 0, 1, 0, 1, 1, 0, 1, 1]])

##### - priority categories
$$r_{cp}: Priority\ value\ of\ campaign\ c\ regarding\ priority\ type\ p\$$

In [5]:
np.ones((C, P), dtype='int8').shape

(10, 10)

In [6]:
r_p = np.random.choice(100, P)
#r_p = np.ones(P, dtype='int8')
rp_c = [r_p[r] for r in np.random.choice(P, C)]
rp_c

[11, 76, 99, 11, 11, 33, 37, 37, 99, 51]

##### - blokage
$$b: Communication\ limit\ per\ person\ for\ the\ whole\ period\$$

In [7]:
b = 7

##### - daily blokage
$$k: Communication\ limit\ per\ person\ at\ each\ day\$$

In [8]:
k = 2

##### - campaign blockage
$$l_c: Communication\ limit\ per\ person\ for\ campaign\ c\$$

In [9]:
l_c = np.random.choice([2,3,4],C)
l_c

array([3, 3, 3, 2, 4, 4, 2, 2, 4, 4])

##### - quota limitations daily/weekly
$$
m_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\
$$
$$
n_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\ each\ day\
$$

In [10]:
m_i = np.random.choice([10,15,18],I)
n_i = np.random.choice([2,3,4],I)
#m_i = np.ones((I), dtype='int8')*10
#n_i = np.ones((I), dtype='int8')*10
(m_i, n_i)

(array([18, 18, 18]), array([3, 4, 3]))

#### - capacity for channel
$$
t_{h,d}: Capacity\ for\ channel\ h\ at\ day\ d.\
$$

In [11]:
t_hd = np.random.choice([700], (H, D))

### Model

In [12]:
mdl = Model(name='Campaign Optimization')

#### Variables
$$
X_{cuhd}\left\{\begin{array}\\
        1 & \mbox{if } Campaign\ c\ will\ be\ sent\ to\ customer\ u\ through\ Channel\ h\ at\ Day\ d \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [13]:
X_cuhd = {(c,u,h,d): mdl.binary_var(f"X_c:{c}_u:{u}_h:{h}_d:{d}")
            for c in range(0,C)
            for u in range(0,U) 
            for h in range(0,H)
            for d in range(0,D)}
"Done"

'Done'

## Maximize
$$\sum_{p \in P}\sum_{c \in C}\sum_{u \in U}\sum_{h \in H}\sum_{d \in D}\,X_{cuhd}\ r_{cp}$$

In [14]:
mdl.maximize(mdl.sum([X_cuhd[(c,u,h,d)] * rp_c[c]
                  for p in range(0,P)
                  for c in range(0,C)
                  for u in range(0,U) 
                  for h in range(0,H) 
                  for d in range(0,D)]))

## subject to

##### - eligibility (2)

$$
X_{cuhd}  \leq e_{cu},\hspace{35pt} \forall c \in C,\forall u \in U,\forall h \in H,\forall d \in D
$$

In [15]:
mdl.add_constraints(
    (X_cuhd[(c,u,h,d)] <= e_cu[c,u]
    for c in range(0,C)
    for u in range(0,U) 
    for h in range(0,H) 
    for d in range(0,D))
)
"Done"

'Done'

##### - use one channel (3)
$$
\sum_{h}X_{cuhd} \le 1,\hspace{35pt} \forall c \in C \, \forall u \in U,\forall d \in D
$$

In [16]:
mdl.add_constraints(
    ((mdl.sum(X_cuhd[(c,u,h,d)] for h in range(0,H)) <= 1)
    for c in range(0,C)
    for u in range(0,U) 
    for d in range(0,D))
)
"Done"

'Done'

##### - weekly communication limitation (4)
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in D} X_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$

In [17]:
mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)] 
               for d in range(0,D) 
               for c in range(0,C) 
               for h in range(0,H)) <= b)
        for u in range(0,U)))
"Done"

'Done'

##### - daily communication limitation (5)
$$
\sum_{h \in H}\sum_{c \in C} X_{cuhd}\le k,\hspace{35pt} \forall u \in U, \forall d \in D
$$

In [18]:
mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]  
               for c in range(0,C) 
               for h in range(0,H)) <= k)
        for d in range(0,D)
        for u in range(0,U)))
"Done"

'Done'

##### - campaign communication limit(6)
$$
\sum_{d \in D}\sum_{h \in H} X_{cuhd}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$

In [19]:
mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]  
               for h in range(0,H) 
               for d in range(0,D)) <= l_c[c] )
        for c in range(0,C)
        for u in range(0,U)))
"Done"

'Done'

##### - weekly quota(7)
$$
\sum_{d \in D}\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$

In [20]:
mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]*q_ic[i,c]
               for c in range(0,C)
               for h in range(0,H) 
               for d in range(0,D)) <= m_i[i])
        for u in range(0,U)
        for i in range(0,I)))
"Done"

'Done'

##### - daily quota(8)
$$
\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le n_i,\hspace{35pt} \forall u \in U,\, \forall d \in D, \forall i \in I
$$

In [21]:
mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]*q_ic[i,c]
               for c in range(0,C) 
               for h in range(0,H)) <= n_i[i])
        for u in range(0,U)
        for d in range(0,D)
        for i in range(0,I)))
"Done"

'Done'

##### Channel capacity (9)
$$
\sum_{c \in C}\sum_{u \in U}{X_{cuhd}}\le t_{hd},\hspace{35pt} \forall d \in D,\, \forall h \in H
$$

##### Binary variable (10)
$$
X_{cuhd} \in \{ 1,0 \},\hspace{35pt} \forall c \in C ,\forall u \in U,\forall d \in D, \forall h \in H
$$

In [22]:
mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]
               for u in range(0,U) 
               for c in range(0,C)) <= t_hd[h,d])
        for h in range(0,H)
        for d in range(0,D)))
"Done"

'Done'

### Solution

In [23]:
solution = mdl.solve(log_output=True)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              202001241
Found incumbent of value 0.000000 after 0.37 sec. (71.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 2743380 rows and 1048677 columns.
Reduced MIP has 476641 rows, 1051323 columns, and 5234313 nonzeros.
Reduced MIP has 1051323 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 12.21 sec. (5150.87 ticks)
Probing time = 3.13 sec. (187.28 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Elapsed time for symmetry detection = 26.28 sec. (10000.20 ticks)
Elapsed time for symmetry detection = 75.84 sec. (20000.24 ticks)
Elapsed time for symmetry detection = 133.62 sec. (30004.36 ticks)
Elapsed time for symmetry detection = 188.58 sec. (40004.36 ticks)
Elapsed time for symmetry detection = 244.69 sec. (50009.94 ticks)
Elapsed time for symmetry detection = 303.40 sec. (60009.94 ticks)
Elapsed time for 

In [24]:
solution

docplex.mp.solution.SolveSolution(obj=1.4553e+07,values={X_c:2_u:1_h:0_d..

In [25]:
print(solution)

solution for: Campaign Optimization
objective: 1.4553e+07
X_c:2_u:1_h:0_d:2=1
X_c:2_u:2_h:2_d:4=1
X_c:2_u:6_h:0_d:0=1
X_c:2_u:6_h:1_d:2=1
X_c:2_u:13_h:0_d:4=1
X_c:2_u:17_h:1_d:6=1
X_c:2_u:17_h:2_d:3=1
X_c:2_u:21_h:0_d:4=1
X_c:2_u:21_h:2_d:5=1
X_c:2_u:22_h:1_d:5=1
X_c:2_u:22_h:2_d:4=1
X_c:2_u:24_h:2_d:4=1
X_c:2_u:25_h:1_d:2=1
X_c:2_u:25_h:1_d:3=1
X_c:2_u:27_h:0_d:3=1
X_c:2_u:31_h:0_d:1=1
X_c:2_u:31_h:1_d:6=1
X_c:2_u:33_h:2_d:2=1
X_c:2_u:34_h:0_d:3=1
X_c:2_u:38_h:2_d:0=1
X_c:2_u:38_h:2_d:2=1
X_c:2_u:45_h:2_d:3=1
X_c:2_u:45_h:2_d:4=1
X_c:2_u:45_h:2_d:5=1
X_c:2_u:46_h:0_d:0=1
X_c:2_u:46_h:2_d:1=1
X_c:2_u:46_h:2_d:4=1
X_c:2_u:48_h:0_d:0=1
X_c:2_u:48_h:0_d:5=1
X_c:2_u:51_h:2_d:0=1
X_c:2_u:58_h:2_d:5=1
X_c:2_u:59_h:0_d:1=1
X_c:2_u:59_h:0_d:4=1
X_c:2_u:59_h:2_d:5=1
X_c:2_u:64_h:1_d:1=1
X_c:2_u:64_h:2_d:3=1
X_c:2_u:64_h:2_d:5=1
X_c:2_u:65_h:1_d:3=1
X_c:2_u:67_h:1_d:4=1
X_c:2_u:75_h:1_d:0=1
X_c:2_u:75_h:2_d:2=1
X_c:2_u:78_h:1_d:4=1
X_c:2_u:78_h:2_d:3=1
X_c:2_u:79_h:1_d:3=1
X_c:2_u:82_h:1_d:5=1
X